In [2]:
!pip install tqdm

In [3]:
# 1. 패키지 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib
from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
# 2. 파일 경로 및 주요 파라미터 별도 선언
승하차_파일 = "../../data/결과/승하차/통합/6호선_승하차인원_통합.csv"
혼잡도_파일 = "../../data/결과/혼잡도/통합/6호선_혼잡도_통합.csv"
정원 = 2000  # 열차 정원




In [5]:
# 3. 데이터 불러오기 및 0패딩 맞춤
def fix_timecols(cols):
    return [c if ":" not in c else c.zfill(5) for c in cols]


def load_data():
    승하차_df = pd.read_csv(승하차_파일, encoding="euc-kr")
    혼잡도_df = pd.read_csv(혼잡도_파일, encoding="euc-kr")
    승하차_df.columns = fix_timecols(승하차_df.columns)
    혼잡도_df.columns = fix_timecols(혼잡도_df.columns)
    return 승하차_df, 혼잡도_df



In [6]:

# 4. 상행/하행 방향 데이터 복제
def duplicate_for_directions(승하차):
    dfs = []
    for direction in ["상행", "하행"]:
        tmp = 승하차.copy()
        tmp["방향"] = direction
        dfs.append(tmp)
    return pd.concat(dfs, ignore_index=True)




In [7]:
# 5. 컬럼 표준화
def standardize_columns(df, do_direction=True):
    df["평일주말"] = df["평일주말"].astype(str).str.strip()
    df["구분"] = df["구분"].astype(str).str.strip()
    df["역번호"] = df["역번호"].astype(str).str.strip()
    if do_direction and "방향" in df.columns:
        df["방향"] = df["방향"].astype(str).str.strip()
    return df



In [8]:

# 6. 벡터 연산 기반 학습 데이터 생성
def build_dl_dataset(승하차, 혼잡도, 정원=2000):
    시간컬럼들 = [col for col in 승하차.columns if ":" in col]
    # key 조합 만들기
    승하차["key"] = (
        승하차["역번호"].astype(str)
        + "_"
        + 승하차["방향"].astype(str)
        + "_"
        + 승하차["평일주말"].astype(str)
    )
    혼잡도["key"] = (
        혼잡도["역번호"].astype(str)
        + "_"
        + 혼잡도["구분"].astype(str)
        + "_"
        + 혼잡도["평일주말"].astype(str)
    )
    # 혼잡도에서 필요한 시간만 추출 후 melt
    혼잡도_long = 혼잡도.melt(
        id_vars=["key"], value_vars=시간컬럼들, var_name="time", value_name="혼잡도"
    )
    승하차_long = 승하차.melt(
        id_vars=["key", "역번호", "방향", "평일주말", "구분"],
        value_vars=시간컬럼들,
        var_name="time",
        value_name="승하차인원",
    )
    # merge
    merged = pd.merge(승하차_long, 혼잡도_long, on=["key", "time"], how="inner")
    # feature 생성
    merged["hour"] = merged["time"].str.split(":").str[0].astype(int)
    merged["평일주말_digit"] = merged["평일주말"].map({"평일": 0, "주말": 1})
    merged["상행"] = (merged["방향"] == "상행").astype(int)
    merged["승차"] = (merged["구분"] == "승차").astype(int)
    merged["혼잡도"] = pd.to_numeric(merged["혼잡도"], errors="coerce").fillna(0)
    merged["승하차인원"] = pd.to_numeric(merged["승하차인원"], errors="coerce").fillna(
        0
    )
    merged["target"] = (merged["혼잡도"] * 정원 / 100).astype(int)
    cols = [
        "역번호",
        "hour",
        "평일주말_digit",
        "상행",
        "승차",
        "승하차인원",
        "혼잡도",
        "target",
    ]
    merged = merged[cols].dropna().reset_index(drop=True)
    merged["역번호"] = merged["역번호"].astype(int)
    return merged




In [9]:
# 7. 입력 값 안전하게 받는 함수
def safe_hour_input():
    while True:
        s = input("시간 (HH:MM): ").strip()
        parts = s.split(":")
        if len(parts) == 2 and parts[0].isdigit():
            return int(parts[0]), f"{int(parts[0]):02d}:00"
        print("시간을 올바른 형식(HH:MM)으로 입력하세요.")




In [10]:
# 8. 사용자 예측 함수 개선
def user_predict_smart(승하차_df, 혼잡도_df, scaler, model):
    역명 = input("역명: ").strip()
    호선 = input("호선: ").strip()
    방향 = input("방향 (상행/하행): ").strip()
    평일주말_str = input("요일 (평일/주말): ").strip()
    hour, 시간컬럼 = safe_hour_input()
    평일주말 = 1 if 평일주말_str == "주말" else 0
    상행 = 1 if 방향 == "상행" else 0

    # 역번호 찾기
    후보 = 승하차_df[
        (승하차_df["역명"] == 역명) & (승하차_df["호선"].astype(str) == str(호선))
    ]
    if 후보.empty:
        print(f"해당 역명/호선이 데이터에 없습니다.")
        return
    역번호 = int(후보.iloc[0]["역번호"])

    # 승차/하차 인원 찾기
    def get_people(df, 구분):
        row = df[
            (df["역번호"] == str(역번호))
            & (df["방향"] == 방향)
            & (df["평일주말"] == 평일주말_str)
            & (df["구분"] == 구분)
        ]
        if not row.empty and 시간컬럼 in row.columns:
            try:
                return float(row.iloc[0][시간컬럼])
            except Exception:
                return 0
        return 0

    승차인원 = get_people(승하차_df, "승차")
    하차인원 = get_people(승하차_df, "하차")
    승하차인원 = 승차인원 + 하차인원

    # 혼잡도 찾기
    row_혼 = 혼잡도_df[
        (혼잡도_df["역번호"] == str(역번호))
        & (혼잡도_df["구분"] == 방향)
        & (혼잡도_df["평일주말"] == 평일주말_str)
    ]
    혼잡도_val = 0
    if not row_혼.empty and 시간컬럼 in row_혼.columns:
        try:
            혼잡도_val = float(row_혼.iloc[0][시간컬럼])
        except Exception:
            혼잡도_val = 0

    # 예측
    arr = np.array([[역번호, hour, 평일주말, 상행, 1, 승하차인원, 혼잡도_val]])
    arr_scaled = scaler.transform(arr)
    pred = model.predict(arr_scaled, verbose=0)
    print(f"예측: {호선}호선 {역명}({역번호}) {방향} {평일주말_str} {시간컬럼}")
    print(f" 승하차 인원(입력): {승하차인원}, 혼잡도(%): {혼잡도_val:.1f}")
    print(f" 딥러닝 기반 예측 열차 내 인원: {int(pred[0][0])} 명")




In [11]:
# 9. 전체 데이터 전처리 및 학습

print("> 데이터 로딩 및 전처리...")
승하차, 혼잡도 = load_data()
승하차_expanded = duplicate_for_directions(승하차)
승하차_expanded = standardize_columns(승하차_expanded)
혼잡도 = standardize_columns(혼잡도)
df = build_dl_dataset(승하차_expanded, 혼잡도, 정원=정원)
if df.shape[0] == 0:
    raise ValueError("학습데이터 생성 실패! 컬럼 또는 값 일치 여부 확인 필요.")
print(f"> 학습데이터 shape: {df.shape}")



> 데이터 로딩 및 전처리...
> 학습데이터 shape: (17560800, 8)


In [12]:
# 10. 입력/정규화/트레인테스트 분리
X = df[
    ["역번호", "hour", "평일주말_digit", "상행", "승차", "승하차인원", "혼잡도"]
].values
y = df["target"].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)



In [ ]:
# 11. 딥러닝 모델과 학습 (Input 경고 해결 및 EarlyStopping 추가)
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True
    )
]
model = keras.Sequential(
    [
        keras.Input(shape=(X_train.shape[1],)),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1),
    ]
)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
history = model.fit(
    X_train,
    y_train,
    validation_split=0.1,
    epochs=40,
    batch_size=64,
    verbose=2,
    callbacks=callbacks,
)



Epoch 1/40
197559/197559 - 372s - 2ms/step - loss: 192.9494 - mae: 0.9911 - val_loss: 0.0886 - val_mae: 0.2428
Epoch 2/40
197559/197559 - 1046s - 5ms/step - loss: 0.3546 - mae: 0.3253 - val_loss: 0.0100 - val_mae: 0.0718
Epoch 3/40
197559/197559 - 585s - 3ms/step - loss: 0.2658 - mae: 0.2757 - val_loss: 0.0099 - val_mae: 0.0711
Epoch 4/40
197559/197559 - 936s - 5ms/step - loss: 0.2214 - mae: 0.2431 - val_loss: 0.0145 - val_mae: 0.0886
Epoch 5/40
197559/197559 - 241s - 1ms/step - loss: 0.1981 - mae: 0.2193 - val_loss: 0.0575 - val_mae: 0.2017
Epoch 6/40
197559/197559 - 474s - 2ms/step - loss: 0.1628 - mae: 0.1998 - val_loss: 0.0058 - val_mae: 0.0520
Epoch 7/40
197559/197559 - 404s - 2ms/step - loss: 0.1464 - mae: 0.1854 - val_loss: 0.0240 - val_mae: 0.0906
Epoch 8/40
197559/197559 - 230s - 1ms/step - loss: 0.1326 - mae: 0.1778 - val_loss: 0.0051 - val_mae: 0.0396
Epoch 9/40
197559/197559 - 236s - 1ms/step - loss: 0.1234 - mae: 0.1728 - val_loss: 0.0080 - val_mae: 0.0570
Epoch 10/40
1975

In [ ]:
# 12. 모델/스케일러 저장 (최신 포맷)
model.save("my_dl_model.6line.keras")
joblib.dump(scaler, "my_scaler.6line.pkl")

# 13. 예측 예제 실행
# user_predict_smart(승하차_expanded, 혼잡도, scaler, model)   # ← 실제 실행 시 활성화